## init

In [8]:
import sys 
sys.path.append('pyfiles/')
import classes_loop as cl
# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2

# math stuff
import numpy as np
# visualization
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import distance
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline
# sliders
from ipywidgets import interactive

# for matrix import and display
import pandas as pd
from IPython.display import display_html 
import csv 
import scipy.sparse
######## NOTE: from this point on, order is important. 
# it would be good to make the code more robust, but right
# now, there are ordered dependencies between functions.

# points = cl.ellipse_example(numpts = 6)
# # note: 100 pts causes handling error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
mat = cl.bdmatrix()
print(mat.initmatrix)

# matrix[i][j]
# matrix1 +2 matrix2
# matrix[i][j] = x

[[0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0]]


In [45]:
sparsemat = scipy.sparse.dia_matrix(mat.initmatrix)
print(sparsemat.toarray()[1][1])
np.array(sparsemat.toarray())

1


array([[0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [49]:
sparsemat = sparsemat + sparsemat
sparsemat.toarray()

array([[0, 4, 0, 0, 0, 0, 0, 0],
       [0, 4, 4, 0, 0, 0, 0, 0],
       [0, 0, 4, 4, 0, 0, 0, 0],
       [0, 0, 0, 4, 4, 0, 0, 0],
       [0, 0, 0, 0, 4, 4, 0, 0],
       [0, 0, 0, 0, 0, 4, 4, 0],
       [0, 0, 0, 0, 0, 0, 4, 4],
       [0, 0, 0, 0, 0, 0, 0, 0]])

1. make 0 matrix from np array
2. edit particular positions to 1
3. reduce matrix 
    - +2 addition
    - identify lowest ones in col
    
    
dictionary where keys are indices (columns), values are sets of row indices with 1s (so max returns lowest one)

In [50]:
d = {
    10 : {1,2,3},
    11 : {3,4,5},
    12 : {1,4,5}
}

In [72]:
d[11] |= {6}

In [70]:
d[11].remove(6)
# screams at you if was already 0

KeyError: 6

In [71]:
d[11] -= {6}
# doesn't scream at you

In [65]:
d

{10: {1, 2, 3}, 11: {3, 4, 5}, 12: {1, 4, 5}}

In [75]:
{1,2} ^ {2,3}
# symm diff

{1, 3}

simple effective time improvements
1. "inverse of low func" eats row indices and spits out col inds, so you get col that has input row as lowest one; e.g. func(5)=7 menas that the 7th column has the lowest one in the 5th row. this eliminates loop of columns from start to my column. n^4 -> n^3
Do just this in what I already have to see speedup
Note: you can separately remember birth simplices so you have essential classes at the end

2. clearing: work within dimension right to left, but inside a dimension, from top dim to bottom dim. if you find a death col, you automatically found a birth col for free, so you can set brth col to 0s. 

In [76]:
d = {7 : 5}

In [83]:
# you can tell it what to tell you if not there, ie -1
d.get(7, -1)

5

# Timing thingz

In [90]:
import time

In [93]:
start_time = time.perf_counter()
for _ in range(1000000):
    25**5
print(f"It took this much time: {time.perf_counter() - start_time :.3f} sec")

It took this much time: 0.044 sec
